# Drone Imagery Search System

In [ ]:
# Install required packages (only needed once)
!pip install flask pandas

## 1. Database Setup

In [ ]:
import os
import re
from datetime import datetime
import sqlite3
import pandas as pd
from flask import Flask, request, jsonify, render_template_string
from IPython.display import HTML, display
import threading

def setup_database():
    """Create SQLite database and tables if they don't exist"""
    conn = sqlite3.connect("drone_folders.db")
    cursor = conn.cursor()

    # Create tables
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS professional (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        date TEXT,
        folder_path TEXT
    );
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS personal (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        date TEXT,
        folder_path TEXT
    );
    """)

    # Add coordinates table with explicit creation
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS professional_coordinates (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        professional_name TEXT,
        lat REAL,
        lon REAL,
        sequence INTEGER,
        FOREIGN KEY(professional_name) REFERENCES professional(name)
    );
    """)

    conn.commit()
    conn.close()
    print("Database setup complete - all tables verified")


# Run the setup and display
setup_database()


Database setup complete - all tables verified
Database setup complete - all tables verified

All tables in database:


,name
0,professional
1,sqlite_sequence
2,personal
3,professional_coordinates



Professional table:


,id,name,date,folder_path
0,1,DJI_202410031255_008_ParadisePondBaseWithLeg,2024-10-03,/content/drive/Shareddrives/Drones - CSC 230/1...
1,2,DJI_202410081539_009_ParadisePondBaseWithLeg,2024-10-08,/content/drive/Shareddrives/Drones - CSC 230/1...
2,3,DJI_202410111314_010_ParadisePondBaseWithLeg,2024-10-11,/content/drive/Shareddrives/Drones - CSC 230/1...
3,4,DJI_202410171306_011,2024-10-17,/content/drive/Shareddrives/Drones - CSC 230/1...
4,5,DJI_202410171306_012_ParadisePondBaseWithLeg,2024-10-17,/content/drive/Shareddrives/Drones - CSC 230/1...
5,6,DJI_202410171502_013_ParadisePondBaseWithLeg,2024-10-17,/content/drive/Shareddrives/Drones - CSC 230/1...
6,7,DJI_202410240928_018_ParadisePondBaseWithLeg,2024-10-24,/content/drive/Shareddrives/Drones - CSC 230/1...



Personal table:


,id,name,date,folder_path
0,1,10.16CampusWideDrone,2024-10-16,/content/drive/Shareddrives/Drones - CSC 230/1...
1,2,10.22Bio363GooseCounting,2024-10-22,/content/drive/Shareddrives/Drones - CSC 230/1...
2,3,10.23Bahamas2020Exports,2020-10-23,/content/drive/Shareddrives/Drones - CSC 230/1...
3,4,10.16Tacoma,2024-10-16,/content/drive/Shareddrives/Drones - CSC 230/1...
4,5,10.20ConnecticutRiver,2024-10-20,/content/drive/Shareddrives/Drones - CSC 230/1...
5,6,10.28DavisLawn,2024-10-28,/content/drive/Shareddrives/Drones - CSC 230/1...



Attempting to show professional_coordinates table...
professional_coordinates table exists but is empty


## 2. Data Import from Google Drive

In [ ]:
from google.colab import drive

def mount_google_drive():
    """Mount Google Drive to access files"""
    drive.mount('/content/drive')
    print("Google Drive mounted")

mount_google_drive()

def extract_lat_lon_all(mrk_file):
    """Extract coordinates from MRK file with tab-delimited parsing"""
    coords = []
    try:
        with open(mrk_file, "r") as f:
            for idx, line in enumerate(f):
                # Split on both tabs and commas
                parts = line.replace('\t', ',').split(',')
                parts = [p.strip() for p in parts if p.strip()]

                try:
                    # Find Lat and Lon markers
                    lat_idx = parts.index('Lat') - 1
                    lon_idx = parts.index('Lon') - 1

                    lat = float(parts[lat_idx])
                    lon = float(parts[lon_idx])
                    coords.append((lat, lon, idx))
                except (ValueError, IndexError):
                    continue

    except Exception as e:
        print(f"Error reading {mrk_file}: {e}")
    return coords

def import_folder_data(folder_path):
    """Import folder names and process MRK files"""
    conn = sqlite3.connect("drone_folders.db")
    cursor = conn.cursor()

    inserted_pro = 0
    inserted_coords = 0

    for name in os.listdir(folder_path):
        full_path = os.path.join(folder_path, name)
        if not os.path.isdir(full_path):
            continue

        # PROFESSIONAL FILES
        if name.startswith("DJI_") and "_" in name:
            try:
                parts = name.split("_")
                raw_date = parts[1]  # e.g. '202410171502'
                parsed_date = datetime.strptime(raw_date, "%Y%m%d%H%M").strftime('%Y-%m-%d')

                # Check if already exists
                cursor.execute("SELECT 1 FROM professional WHERE name = ?", (name,))
                if cursor.fetchone() is None:
                    cursor.execute("""
                        INSERT INTO professional (name, date, folder_path)
                        VALUES (?, ?, ?)
                    """, (name, parsed_date, full_path))
                    inserted_pro += 1
                    print(f"✅ Inserted professional folder: {name}")

                # Process MRK file regardless of whether it's new
                mrk_files = [f for f in os.listdir(full_path) if f.lower().endswith(".mrk")]
                if mrk_files:
                    # Clear existing coordinates for this flight (if any)
                    cursor.execute("""
                        DELETE FROM professional_coordinates
                        WHERE professional_name = ?
                    """, (name,))

                    # Process each MRK file found
                    for mrk_file in mrk_files:
                        mrk_path = os.path.join(full_path, mrk_file)
                        coords = extract_lat_lon_all(mrk_path)
                        for seq, (lat, lon, _) in enumerate(coords):
                            cursor.execute("""
                                INSERT INTO professional_coordinates
                                (professional_name, lat, lon, sequence)
                                VALUES (?, ?, ?, ?)
                            """, (name, lat, lon, seq))
                        inserted_coords += len(coords)
                        print(f"  ↳ Processed {len(coords)} coordinates from {mrk_file}")
                else:
                    print(f"  ↳ No MRK files found in {name}")

            except Exception as e:
                print(f"❌ Error processing {name}: {str(e)}")

        # PERSONAL FILES
        elif name.count('.') == 1:
            try:
                date_part = name.split('.')[0]
                day_and_rest = name.split('.')[1]
                day = day_and_rest[:2]
                rest = day_and_rest[2:]

                match = re.search(r'(20\d{2})', rest)
                year = match.group(1) if match else "2024"
                parsed_date = datetime.strptime(f"{year}-{date_part}-{day}", "%Y-%m-%d").strftime('%Y-%m-%d')

                cursor.execute("SELECT 1 FROM personal WHERE name = ?", (name,))
                if cursor.fetchone() is None:
                    cursor.execute("INSERT INTO personal (name, date, folder_path) VALUES (?, ?, ?)",
                                 (name, parsed_date, full_path))
                    inserted_per += 1
                    print(f"✅ Inserted personal folder: {name}")
            except Exception as e:
                print(f"❌ Failed to parse personal folder {name}: {e}")
        else:
            print(f"⚠️ Unrecognized format: {name}")

    conn.commit()
    conn.close()

# Usage: (uncomment and modify path as needed)
folder_path = '/content/drive/Shareddrives/Drones - CSC 230/10-October'
import_folder_data(folder_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted
  ↳ Processed 257 coordinates from DJI_202410031255_008_ParadisePondBaseWithLeg_Timestamp.MRK
  ↳ Processed 257 coordinates from DJI_202410081539_009_ParadisePondBaseWithLeg_Timestamp.MRK
  ↳ Processed 257 coordinates from DJI_202410111314_010_ParadisePondBaseWithLeg_Timestamp.MRK
  ↳ Processed 2 coordinates from DJI_202410171306_011_Timestamp.MRK
  ↳ Processed 257 coordinates from DJI_202410171306_012_ParadisePondBaseWithLeg_Timestamp.MRK
  ↳ Processed 257 coordinates from DJI_202410171502_013_ParadisePondBaseWithLeg_Timestamp.MRK
  ↳ Processed 257 coordinates from DJI_202410240928_018_ParadisePondBaseWithLeg_Timestamp.MRK

✅ Total professional folders inserted: 0


## 3. Run the Application


In [ ]:
from flask import Flask, request, jsonify, render_template_string
from google.colab import output
from IPython.display import HTML, display
import threading
import sqlite3
from datetime import datetime
import time
import socket
from flask import send_from_directory
import os

app = Flask(__name__)

# Database connection
def get_db_connection():
    conn = sqlite3.connect("drone_folders.db")
    conn.row_factory = sqlite3.Row
    return conn

# HTML template
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Drone Imagery Search</title>
    <style>
        body { font-family: Arial, sans-serif; padding: 20px; }
        .search-form { background: #f5f5f5; padding: 20px; border-radius: 8px; margin-bottom: 20px; }
        .form-group { margin-bottom: 15px; }
        label { display: inline-block; width: 100px; font-weight: bold; }
        input, select { padding: 8px; border: 1px solid #ddd; border-radius: 4px; }
        button { background: #4CAF50; color: white; padding: 10px 15px; border: none; border-radius: 4px; cursor: pointer; }
        button:hover { background: #45a049; }
        .results { margin-top: 20px; }
        .result-item { border: 1px solid #ddd; padding: 15px; margin-bottom: 10px; border-radius: 4px; }
        .result-type { font-weight: bold; color: #fff; padding: 3px 8px; border-radius: 4px; }
        .professional { background: #2196F3; }
        .personal { background: #4CAF50; }
        .error { color: red; margin-top: 10px; }
    </style>
</head>
<body>
    <h1>Drone Imagery Search</h1>
    <div class="search-form">
        <form id="searchForm">
            <div class="form-group">
                <label for="date_from">From Date:</label>
                <input type="date" id="date_from" name="date_from" required>
            </div>
            <div class="form-group">
                <label for="date_to">To Date:</label>
                <input type="date" id="date_to" name="date_to" required>
            </div>
            <div class="form-group">
                <label for="imagery_type">Type:</label>
                <select id="imagery_type" name="imagery_type">
                    <option value="all">All</option>
                    <option value="professional">Professional</option>
                    <option value="personal">Personal</option>
                </select>
            </div>
            <button type="submit">Search</button>
        </form>
    </div>
    <div id="results"></div>
    <script>
        document.getElementById('searchForm').addEventListener('submit', async (e) => {
            e.preventDefault();
            const resultsDiv = document.getElementById('results');
            resultsDiv.innerHTML = '<p>Searching...</p>';

            const formData = new FormData(e.target);
            const params = new URLSearchParams(formData);

            try {
                const response = await fetch('/search?' + params.toString());
                if (!response.ok) {
                    throw new Error(`HTTP error! status: ${response.status}`);
                }
                const data = await response.json();

                if (data.error) {
                    resultsDiv.innerHTML = `<div class="error">${data.error}</div>`;
                    return;
                }

                if (data.results.length === 0) {
                    resultsDiv.innerHTML = '<div>No results found for the selected criteria</div>';
                    return;
                }

                let html = '';
                data.results.forEach(item => {
                    html += `
                        <div class="result-item">
                            <h3>${item.name}</h3>
                            <p>Date: ${item.date}</p>
                            <p>Type: <span class="result-type ${item.type}">${item.type}</span></p>
                            <p>Path: ${item.file_path}</p>
                        </div>
                    `;
                });

                resultsDiv.innerHTML = html;
            } catch (error) {
                console.error('Error:', error);
                resultsDiv.innerHTML = '<div class="error">Error fetching results. Check console for details.</div>';
            }
        });
    </script>
</body>
</html>
"""

# Flask routes
@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/search')
def search():
    date_from = request.args.get('date_from')
    date_to = request.args.get('date_to')
    imagery_type = request.args.get('imagery_type', 'all')

    if not date_from or not date_to:
        return jsonify({'error': 'Both date ranges are required'}), 400

    try:
        # Convert dates to datetime.date objects for proper comparison
        date_from = datetime.strptime(date_from, '%Y-%m-%d').date()
        date_to = datetime.strptime(date_to, '%Y-%m-%d').date()
    except ValueError:
        return jsonify({'error': 'Invalid date format. Use YYYY-MM-DD'}), 400

    conn = get_db_connection()
    results = []

    try:
        if imagery_type in ['all', 'professional']:
            professional = conn.execute("""
                SELECT
                    name,
                    date,
                    folder_path as file_path,
                    'professional' as type
                FROM professional
                WHERE date BETWEEN ? AND ?
                ORDER BY date
            """, (date_from, date_to))
            results.extend([dict(row) for row in professional.fetchall()])

        if imagery_type in ['all', 'personal']:
            personal = conn.execute("""
                SELECT
                    name,
                    date,
                    folder_path as file_path,
                    'personal' as type
                FROM personal
                WHERE date BETWEEN ? AND ?
                ORDER BY date
            """, (date_from, date_to))
            results.extend([dict(row) for row in personal.fetchall()])

        return jsonify({'results': results})

    except Exception as e:
        return jsonify({'error': str(e)}), 500
    finally:
        conn.close()

def flight_map():
    """Render the flight map page"""
    return render_template_string("""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Flight Map</title>
        <meta charset="utf-8" />
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.4/dist/leaflet.css" />
        <style>
            #map { height: 600px; }
        </style>
    </head>
    <body>
        <h1>Drone Flight Paths</h1>
        <div id="map"></div>
        <script src="https://unpkg.com/leaflet@1.9.4/dist/leaflet.js"></script>
        <script>
            const map = L.map('map').setView([42.3251, -72.6412], 15);
            L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
                maxZoom: 18,
            }).addTo(map);

            fetch('/api/flights')
                .then(response => response.json())
                .then(data => {
                    for (const [name, coords] of Object.entries(data)) {
                        const polyline = L.polyline(coords, {color: 'blue'}).addTo(map);
                        polyline.bindPopup(`<b>${name}</b>`);
                    }
                })
                .catch(err => console.error('Error loading flight data:', err));
        </script>
    </body>
    </html>
    """)

@app.route('/map')
def show_map():
    """Render the flight map page"""
    return render_template_string("""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Flight Map</title>
        <meta charset="utf-8" />
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.4/dist/leaflet.css" />
        <style>
            body { padding: 20px; }
            #map { height: 600px; margin-top: 20px; }
            .back-link { margin-bottom: 20px; display: inline-block; }
        </style>
    </head>
    <body>
        <a class="back-link" href="/">← Back to Search</a>
        <h1>Drone Flight Paths</h1>
        <div id="map"></div>

        <script src="https://unpkg.com/leaflet@1.9.4/dist/leaflet.js"></script>
        <script>
            const map = L.map('map').setView([42.3251, -72.6412], 15);
            L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
                maxZoom: 18,
                attribution: '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
            }).addTo(map);

            fetch('/api/flights')
                .then(response => {
                    if (!response.ok) {
                        throw new Error('Network response was not ok');
                    }
                    return response.json();
                })
                .then(data => {
                    if (Object.keys(data).length === 0) {
                        const infoDiv = document.createElement('div');
                        infoDiv.innerHTML = '<p>No flight path data available</p>';
                        document.body.appendChild(infoDiv);
                        return;
                    }

                    for (const [name, coords] of Object.entries(data)) {
                        if (coords.length > 0) {
                            const polyline = L.polyline(coords, {color: 'blue'}).addTo(map);
                            polyline.bindPopup(`<b>${name}</b>`);
                            // Auto-fit the map to the bounds of all polylines
                            map.fitBounds(polyline.getBounds());
                        }
                    }
                })
                .catch(err => {
                    console.error('Error loading flight data:', err);
                    const errorDiv = document.createElement('div');
                    errorDiv.innerHTML = `<p style="color:red">Error loading flight data: ${err.message}</p>`;
                    document.body.appendChild(errorDiv);
                });
        </script>
    </body>
    </html>
    """)


@app.route('/api/flights')
def get_flights():
    """API endpoint to get flight path data"""
    conn = get_db_connection()
    try:
        # Get all professional flights with coordinates
        flights = conn.execute("""
            SELECT p.name, pc.lat, pc.lon, pc.sequence
            FROM professional p
            JOIN professional_coordinates pc ON p.name = pc.professional_name
            ORDER BY p.name, pc.sequence
        """).fetchall()

        # Group coordinates by flight name
        flight_data = {}
        for row in flights:
            name = row['name']
            if name not in flight_data:
                flight_data[name] = []
            flight_data[name].append([row['lat'], row['lon']])

        return jsonify(flight_data)
    except Exception as e:
        return jsonify({'error': str(e)}), 500
    finally:
        conn.close()

# Shutdown function
def shutdown_server():
    func = request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    func()

@app.route('/shutdown', methods=['POST'])
def shutdown():
    shutdown_server()
    return 'Server shutting down...'


def find_free_port():
    """Find a free port starting from 9000"""
    port = 9000
    while True:
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.bind(('0.0.0.0', port))
                return port
        except OSError:
            port += 1

def run_flask(port):
    app.run(host='0.0.0.0', port=port, debug=False, use_reloader=False)

# Find an available port
port = find_free_port()

# Start Flask in a thread
flask_thread = threading.Thread(target=run_flask, args=(port,))
flask_thread.daemon = True
flask_thread.start()

# Wait for server to start
time.sleep(2)

# Get proxy URL
proxy_url = output.eval_js(f'google.colab.kernel.proxyPort({port})')

# Display interface with FIXED shutdown button
display(HTML(f"""
    <h2>Drone Imagery Search Interface</h2>
    <div style="margin-bottom: 15px;">
        <a href="{proxy_url}" target="_blank" style="margin-right: 15px;">Open Search</a>
        <a href="{proxy_url}/map" target="_blank">Open Flight Map</a>
    </div>
    <iframe src="{proxy_url}" width="100%" height="600px"></iframe>
    <p>To stop the server: <button onclick="fetch('{proxy_url}/shutdown', {{'method': 'POST'}})">Shutdown</button></p>
    <p>Running on port: {port}</p>
"""))
print(f"If the interface doesn't load, visit: {proxy_url}")

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:9006
 * Running on http://172.28.0.12:9006
INFO:werkzeug:Press CTRL+C to quit


If the interface doesn't load, visit: https://9006-m-s-3k7vykytdwftl-b.us-east1-0.prod.colab.dev


## 4. View database content (for debugging purposes)

In [ ]:
def show_database_contents():
    """Display database contents for verification"""
    # First ensure tables exist
    setup_database()

    conn = sqlite3.connect("drone_folders.db")

    print("\nAll tables in database:")
    tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
    display(tables.reset_index(drop=True))  # Add reset_index() to avoid reference issues

    print("\nProfessional table (first 10 entries):")
    df_pro = pd.read_sql_query("SELECT * FROM professional LIMIT 10", conn)
    display(df_pro.reset_index(drop=True))

    print("\nPersonal table (first 10 entries):")
    df_per = pd.read_sql_query("SELECT * FROM personal LIMIT 10", conn)
    display(df_per.reset_index(drop=True))

    try:
        print("\nProfessional_coordinates table status:")
        # First check if table has any data
        count = pd.read_sql_query("SELECT COUNT(*) as count FROM professional_coordinates", conn)
        display(count)

        if count.iloc[0]['count'] > 0:
            print("\nSample coordinate data (first 20 points):")
            df_coords = pd.read_sql_query("""
                SELECT
                    pc.id,
                    pc.professional_name,
                    p.date,
                    pc.lat,
                    pc.lon,
                    pc.sequence
                FROM professional_coordinates pc
                LEFT JOIN professional p ON pc.professional_name = p.name
                ORDER BY pc.professional_name, pc.sequence
                LIMIT 20
            """, conn)
            display(df_coords.reset_index(drop=True))

            print("\nFlights with coordinate data:")
            df_counts = pd.read_sql_query("""
                SELECT
                    professional_name,
                    COUNT(*) as point_count
                FROM professional_coordinates
                GROUP BY professional_name
                ORDER BY professional_name
            """, conn)
            display(df_counts.reset_index(drop=True))
        else:
            print("The professional_coordinates table is empty.")
            print("\nTo populate it, you need to:")
            print("1. Ensure your professional folders contain .MRK files")
            print("2. Re-run the import_folder_data() function")
            print("3. Verify the MRK files contain coordinate data in the format:")
            print("   LATITUDE,Lat,LONGITUDE,Lon,...")

    except Exception as e:
        print(f"\nError accessing professional_coordinates table: {str(e)}")
        print("\nTry these troubleshooting steps:")
        print("1. Run setup_database() to ensure tables exist")
        print("2. Check if the table exists with:")
        print("   pd.read_sql_query(\"SELECT name FROM sqlite_master WHERE type='table' AND name='professional_coordinates'\", conn)")

    conn.close()

# Run the display
show_database_contents()

Database setup complete - all tables verified

All tables in database:


,name
0,professional
1,sqlite_sequence
2,personal
3,professional_coordinates



Professional table (first 10 entries):


,id,name,date,folder_path
0,1,DJI_202410031255_008_ParadisePondBaseWithLeg,2024-10-03,/content/drive/Shareddrives/Drones - CSC 230/1...
1,2,DJI_202410081539_009_ParadisePondBaseWithLeg,2024-10-08,/content/drive/Shareddrives/Drones - CSC 230/1...
2,3,DJI_202410111314_010_ParadisePondBaseWithLeg,2024-10-11,/content/drive/Shareddrives/Drones - CSC 230/1...
3,4,DJI_202410171306_011,2024-10-17,/content/drive/Shareddrives/Drones - CSC 230/1...
4,5,DJI_202410171306_012_ParadisePondBaseWithLeg,2024-10-17,/content/drive/Shareddrives/Drones - CSC 230/1...
5,6,DJI_202410171502_013_ParadisePondBaseWithLeg,2024-10-17,/content/drive/Shareddrives/Drones - CSC 230/1...
6,7,DJI_202410240928_018_ParadisePondBaseWithLeg,2024-10-24,/content/drive/Shareddrives/Drones - CSC 230/1...



Personal table (first 10 entries):


...

,count
0,1544



Sample coordinate data (first 20 points):


,id,professional_name,date,lat,lon,sequence
0,1545,DJI_202410031255_008_ParadisePondBaseWithLeg,2024-10-03,42.319500,-72.641381,0
1,1546,DJI_202410031255_008_ParadisePondBaseWithLeg,2024-10-03,42.319375,-72.641378,1
2,1547,DJI_202410031255_008_ParadisePondBaseWithLeg,2024-10-03,42.319253,-72.641375,2
3,1548,DJI_202410031255_008_ParadisePondBaseWithLeg,2024-10-03,42.319136,-72.641372,3
4,1549,DJI_202410031255_008_ParadisePondBaseWithLeg,2024-10-03,42.319014,-72.641369,4
5,1550,DJI_202410031255_008_ParadisePondBaseWithLeg,2024-10-03,42.318898,-72.641367,5
6,1551,DJI_202410031255_008_ParadisePondBaseWithLeg,2024-10-03,42.318781,-72.641365,6
7,1552,DJI_202410031255_008_ParadisePondBaseWithLeg,2024-10-03,42.318665,-72.641362,7
8,1553,DJI_202410031255_008_ParadisePondBaseWithLeg,2024-10-03,42.318548,-72.641358,8
9,1554,DJI_202410031255_008_ParadisePondBaseWithLeg,2024-10-03,42.318430,-72.641355,9



Flights with coordinate data:


,professional_name,point_count
0,DJI_202410031255_008_ParadisePondBaseWithLeg,257
1,DJI_202410081539_009_ParadisePondBaseWithLeg,257
2,DJI_202410111314_010_ParadisePondBaseWithLeg,257
3,DJI_202410171306_011,2
4,DJI_202410171306_012_ParadisePondBaseWithLeg,257
5,DJI_202410171502_013_ParadisePondBaseWithLeg,257
6,DJI_202410240928_018_ParadisePondBaseWithLeg,257
